In [1]:
!pip install yappi

In [2]:
%run common_init.py

Start import
calculating effective efficiency for Xe detector done
calculating effective efficiency for Ge detector done
calculating effective efficiency for Ar detector done
calculating effective efficiency for Xe_migd detector done
calculating effective efficiency for Ge_migd detector done
calculating effective efficiency for Ar_migd detector done
Done import
SYSTEM
	Running on linux
	Python version 3.7.4 (default, Aug 13 2019, 20:35:49) [GCC 7.3.0]
	Python installation /home/joran/.conda/envs/DD_DM/bin/python
MODULES
	DirectDmTargets
		ver.:	0.2.0
		Path:	/mnt/c/Users/Joran/Google Drive/PhD-master/ubuntu-storage/windows-anaconda/DD_DM_targets/DirectDmTargets/

	wimprates
		ver.:	0.3.2
		Path:	/mnt/c/Users/Joran/Google Drive/PhD-master/ubuntu-storage/windows-anaconda/forked_wimprates/wimprates/



In [3]:
# import yappi
# filename = 'test.prof'
# yappi.set_clock_type('cpu')
# yappi.start(builtins=True)
dddm.NestleStatModel('Xe_migd')
# stats = yappi.get_func_stats()
# stats.save(filename, type='callgrind')

StatModel::	2020-01-07T10:49
	initialized for Xe_migd detector.See print(stat_model) for default settings
check_save_folder::	making ../../verne/results/veldists/f_params/loc_XENON/v0_230/vesc_543
check_save_folder::	making ../../verne/results/veldists/f_params/loc_XENON/v0_230/vesc_543/rho_0.400
No spectrum found at:
../../verne/results/veldists/f_params/loc_XENON/v0_230/vesc_543/rho_0.400/sig_-45.0_mx_1.70_avg.csv
Generating spectrum, this can take a minute. Execute:
python "../../verne/"/src/CalcVelDist.py -m_x 49.99999999999999 -sigma_p 1e-45 -loc XENON -path "../../verne//src/" -v_0 230.0 -v_esc 543.9999999999999 -save_as "../../verne/results/veldists/f_params/loc_XENON/v0_230/vesc_543/rho_0.400/sig_-45.0_mx_1.70_avg.csv" 


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/joran/.conda/envs/DD_DM/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-d9c314f263b9>", line 5, in <module>
    dddm.NestleStatModel('Xe_migd')
  File "/mnt/c/Users/Joran/Google Drive/PhD-master/ubuntu-storage/windows-anaconda/DD_DM_targets/DirectDmTargets/nestle.py", line 26, in __init__
    StatModel.__init__(self, *args)
  File "/mnt/c/Users/Joran/Google Drive/PhD-master/ubuntu-storage/windows-anaconda/DD_DM_targets/DirectDmTargets/statistics.py", line 97, in __init__
    self.set_default()
  File "/mnt/c/Users/Joran/Google Drive/PhD-master/ubuntu-storage/windows-anaconda/DD_DM_targets/DirectDmTargets/statistics.py", line 197, in set_default
    self.eval_benchmark()
  File "/mnt/c/Users/Joran/Google Drive/PhD-master/ubuntu-storage/windows-anaconda/DD_DM_targets/DirectDmTargets/statistics.py", line 219, in eval_benchmar

NameError: name 'memoize' is not defined

In [ ]:
%%time
dddm.NestleStatModel('Xe_migd')

In [ ]:
%%time
dddm.NestleStatModel('Xe_migd')

In [ ]:
%%time
dddm.NestleStatModel('Xe_migd')


In [ ]:
%%time
spec = dddm.VerneSHM()
test = dddm.DetectorSpectrum(1, 1e-40, spec, dddm.experiment['Xe_migd'])

test.get_data(poisson = False)

In [ ]:
%%time
# This is with setting the dblquad to a lower precision
spec = dddm.VerneSHM()
test = dddm.DetectorSpectrum(1, 1e-40, spec, dddm.experiment['Xe_migd'])
test.get_data(poisson = False)

In [ ]:
%%time
# This is with setting the dblquad to a default precision
spec = dddm.VerneSHM()
test = dddm.DetectorSpectrum(1, 1e-40, spec, dddm.experiment['Xe_migd'])
test.get_data(poisson = False)

In [ ]:
import DirectDmTargets as dddm

In [ ]:
import wimprates as wr

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import numericalunits as nu

In [ ]:
from tqdm import tqdm

In [ ]:
from scipy.integrate import quad as scipy_int

# Direct detection of Dark matter using different target materials #

Author:

Joran Angevaare <j.angevaare@nikef.nl>

Date:

14 october 2019 

## Goal ## 

- Roughly reproduce <https://journals.aps.org/prd/abstract/10.1103/PhysRevD.83.083505>
- Update the results thereof with more recent knowledge of the DM-distribution

### Approach ###
To achieve these goals, we must first get a decent recoil spectrum, that is flexible enough to have different astrophysical parameters. Further, it must also be flexible enough to be able to allow for different analytic functions to be added to the model. For instance, we must be able to incorporate the $\rm{SHM}^{++}$ as presented here <https://arxiv.org/abs/1810.11468>.

When we have a sufficiently flexible model, we want to add in the detector physics, we should therein incorporate at least the following parameters:
- target
  - material
  - cross-section
- threshold
- background
- cut efficiency  
- volume
- exposure

Finally we should perform the inference

In [ ]:
dddm.experiment['Xe']['res'](1)

## Detector smearing

## Distribution of the DM ##
First we need to make a DM-rate spectrum

In [ ]:
use_SHM = dddm.SHM()

In [ ]:
def plot_spectrum(data, color = 'blue', label = 'label'):
    plt.errorbar(data['bin_centers'], data['counts'], 
                xerr=(data['bin_left'] - data['bin_right'])/2,
                yerr = np.sqrt(data['counts']),
                color = color,
                linestyle = 'none',
                capsize = 2,
                marker = 'o',
                label = label
                )

In [ ]:
%%time
xe_events = dddm.GenSpectrum(50, 1e-45, use_SHM, dddm.experiment['Xe'])
xe_events.n_bins = 10
xe_galact = xe_events.get_data(poisson = False)
xe_events.n_bins

In [ ]:
%%time
xe_events = dddm.DetectorSpectrum(50, 1e-45, use_SHM, dddm.experiment['Xe'])
xe_events.n_bins = 10

xe_det = xe_events.get_data(poisson = False)

In [ ]:
plot_spectrum(xe_galact, label = 'un-smeared')
plot_spectrum(xe_det, 'red', label = 'det-resolution')
# plt.yscale("log")
plt.legend()

In [ ]:
from __future__ import print_function
from IPython.core import page
page.page = print

In [ ]:
!pip install line_profiler --user

In [ ]:
%load_ext line_profiler

In [ ]:
test_class = dddm.DetectorSpectrum(50, 1e-45, use_SHM, dddm.detectors['Xe'])
test_class.n_bins = 3
test_class.n_bins_result = 15

In [ ]:
test = test_class.get_data
%lprun -f test test(poisson = False)

In [ ]:
test = test_class.get_events
%lprun -f test test()

In [ ]:
test = test_class.compute_detected_spectrum
test_class.n_bins = 10
test_class.n_bins_result = 100
%lprun -f test test()

In [ ]:
test = test_class.spectrum_simple
test_class.n_bins = 10
test_class.n_bins_result = 100
%lprun -f test test([50, 1e-45])

In [ ]:
test_2 = wr.rate_wimp_std
%lprun -f test_2 test_2(np.linspace(1,100,10), 50, 1e-45)

In [ ]:
test_2 = wr.rate_wimp
%lprun -f test_2 test_2(np.linspace(1,100,10), 50, 1e-45)